# How climate and envrionmental justice tools are used in research

studies

Elham Ali (Public Environmental Data Project)  
October 4, 2025

TBD

## Background

When public climate & EJ evidence disappears (removed, restricted, or altered), a decade of downstream knowledge becomes harder to verify, reproduce, teach, or apply—especially for communities and decisions that most need it.

This analysis looks at how many studies have used these tools, their topics, and their use cases.

## Questions

Here are the key questions explored in this analysis:

-   How many research papers or studies have cited or relied on five U.S. federal climate and environmental tools that are threatened, not being maintained, or no longer available?
-   What research topics are associated with each tool, and how do these topics vary by citation frequency?
-   In what ways are these tools applied in research?

## Data Sources

The climate tools assessed are:

-   CEQ’s **Climate and Economic Justice Screening Tool (CEJST)**
-   EPA’s **EJScreen**
-   USDA Forest Service’s **Climate Risk Viewer (CRV)**
-   FEMA’s **Future Risk Index (FRI)**
-   CDC’s **Environmental Justice Index (EJI)**

The data for this project comes from Google scholar and last refreshed on September 15, 2025.

Original raw datasets are saved in the `data/` folder. This script reduces and cleans those datasets to prepare them for analysis.

## Cleaning

I start by loading the packages needed for file handling, data wrangling, and visualization.

In [ ]:
## Folder structure helpers
library(here)

here() starts at /Users/elhamali/Documents/Data Projects/climate-ej-tools-publications

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.1.0
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'janitor'

The following objects are masked from 'package:stats':

    chisq.test, fisher.test

Loading required package: grid

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Highcharts (www.highcharts.com) is a Highsoft software product which is
not free for commercial and Governmental use


Attaching package: 'igraph'

The following objects are masked from 'package:lubridate':

    %--%, union

The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union

The following objects are masked from 'package:purrr':

    compose, simplify

The following object is masked from 'package:tidyr':

    crossing

The following object is masked from 'package:tibble':

    as_data_frame

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

### Import raw data

I import all .csv files from the `01_raw_data/` folder, then save them as .rds files into `05_cleaned_datasets/`. This preserves their structure and speeds up future reads.

In [ ]:
# List all CSV files
csv_files <- list.files(here("01_raw_data"), pattern = "\\.csv$", full.names = TRUE)

# Read into a list of dataframes
datasets <- map(csv_files, read.csv)
names(datasets) <- tools::file_path_sans_ext(basename(csv_files))

# Save each dataset as .rds in 05_cleaned_datasets/
walk2(
  datasets,
  names(datasets),
  ~ saveRDS(.x, here("05_cleaned_datasets", paste0(.y, ".rds")))
)

### Merge EJScreen datasets

Google Scholar caps search results at 1,000 records \[@harzing2025\].

Because EJScreen returned more than this, I split the results into four files (ejscreen_1–ejscreen_4). Here I combine them into one dataframe.

In [ ]:
# List EJScreen RDS files
ejscreen_files <- here("05_cleaned_datasets") |>
  list.files(pattern = "^ejscreen_\\d+\\.rds$", full.names = TRUE)

# Read, tag, clean variable names
ejscreen_list <- map(ejscreen_files, ~ {
  readRDS(.x) |>
    mutate(source_file = tools::file_path_sans_ext(basename(.x))) |>
    janitor::clean_names()
})

# Merge into one dataframe
ejscreen_all <- bind_rows(ejscreen_list)

# Quick check on dimensions
dim(ejscreen_all)  

[1] 1258   27

### Clean and tag all other tools

I apply the same cleaning process to the other tools (CEJST, CRV, EJI, FRI):

-   Standardize variable names to snake_case
-   Add a source_file column for provenance
-   Guess variable types (integers, doubles, dates, etc.)

In [ ]:
# Helper function: read, clean names, add provenance, type-convert
read_clean_tag <- function(path) {
  df <- readRDS(path) |>
    janitor::clean_names()

  source_stem <- tools::file_path_sans_ext(basename(path))  # e.g., "ejscreen_1" or "cejst"

  df |>
    mutate(
      source_file = source_stem,
      tool_name   = sub("_\\d+$", "", source_stem)  # "ejscreen_1" -> "ejscreen"; "cejst" -> "cejst"
    ) |>
    readr::type_convert(col_types = readr::cols(.default = readr::col_guess())) |>
    mutate(across(where(is.character), trimws))
}
# 
# # Rebuild ejscreen_all if needed
# if (!exists("ejscreen_all")) {
#   ejs_files <- list.files(here("05_cleaned_datasets"), pattern = "^ejscreen_\\d+\\.rds$", full.names = TRUE)
#   ejscreen_all <- ejs_files |> map(read_clean_tag) |> list_rbind()
# }

# Process other tools
other_tools <- c("cejst", "crv", "eji", "fri")
other_paths <- here("05_cleaned_datasets", paste0(other_tools, ".rds"))

other_list <- other_paths |>
  set_names(~ tools::file_path_sans_ext(basename(.x))) |>
  map(read_clean_tag)

### Final merge and labeling

I now merge all tools into a single dataset. Additional steps:

-   Ensure consistent date formats
-   Map tool codes (ejscreen, cejst, etc.) to their full names
-   Drop empty placeholder columns

In [ ]:
# Helper: coerce anything date-like into Date
to_Date <- function(x) {
  if (inherits(x, "Date"))    return(x)
  if (inherits(x, "POSIXt"))  return(as_date(x))
  if (is.numeric(x))          return(as_date(as.POSIXct(x, origin = "1970-01-01", tz = "UTC")))
  if (is.character(x)) {
    suppressWarnings({
      dt <- ymd_hms(x, quiet = TRUE, tz = "UTC")
      idx <- is.na(dt); if (any(idx)) dt[idx] <- ymd(x[idx], quiet = TRUE)
      idx <- is.na(dt); if (any(idx)) dt[idx] <- mdy(x[idx], quiet = TRUE)
      idx <- is.na(dt); if (any(idx)) dt[idx] <- dmy(x[idx], quiet = TRUE)
    })
    return(as_date(dt))
  }
  as_date(as.character(x))
}

# Apply date coercion
if (exists("ejscreen_all") && "query_date" %in% names(ejscreen_all)) {
  ejscreen_all <- ejscreen_all %>% mutate(query_date = to_Date(query_date))
}
if (exists("other_list")) {
  other_list <- other_list %>%
    map(~ if ("query_date" %in% names(.x)) mutate(.x, query_date = to_Date(query_date)) else .x)
} else {
  other_list <- list()
}

# Merge all datasets
all_tools <- bind_rows(ejscreen_all, !!!other_list)

# Map tool codes to full names
all_tools <- all_tools %>%
  mutate(
    source_file = if_else(is.na(source_file) | source_file == "",
                          coalesce(source_file, tool_name), source_file),
    base_tool   = sub("_\\d+$", "", source_file),   # ejscreen_1 -> ejscreen
    tool_name   = dplyr::recode(
      base_tool,
      ejscreen = "EJScreen",
      cejst    = "Climate and Economic Justice Screening Tool (CEJST)",
      crv      = "Climate Risk Viewer",
      fri      = "Future Risk Index (FRI)",
      eji      = "Environmental Justice Index (EJI)",
      .default = tool_name  # keep whatever was there if not matched
    )
  ) %>%
  select(-base_tool)

# Drop empty placeholder columns
all_tools <- all_tools %>%
  select(-any_of(c("issn", "citation_url", "volume", "issue", "start_page", "end_page")))

# Save combined dataset
saveRDS(all_tools, here("05_cleaned_datasets", "all_tools.rds"))
write_csv(all_tools, here("05_cleaned_datasets", "all_tools.csv"))

## Analysis

### Stratified random sampling

A collaborator to double-code a subset of research studies. Together, we will manually assign each study to a **topic category** and a **tool use type**. This step establishes **inter-rater agreement** (agreement between coders beyond chance).

For sampling, I am taking **10% of the full dataset (2,170 studies) = 217 studies**. Sampling will be stratified by tool `tool_name`. The entire 222-study subset will be double-coded.

A helpful guide is *Stratified Sampling in R: A Practical Guide with Base R and dplyr* \[@sanderson2024\].

I considered two allocation approaches:

1.  **Proportional-only allocation** (222 total).

    -   Each tool receives samples proportional to its share of the corpus.

    -   This preserves representativeness but leaves small tools (e.g., CRV, FRI) with too few cases to analyze.

2.  **Proportional with a minimum floor** (222 total, MIN = 10).

    -   Every tool gets at least 10 studies (or all available, if fewer).

    -   Larger tools (EJScreen, CEJST) give up a small number of cases to make room.

I chose the **minimum floor approach**, since it ensures smaller tools are represented enough for meaningful analysis and coder agreement checks.

**Tradeoff:**

-   *Proportional-only*: best if the goal is a population-level snapshot but leaves tiny strata nearly invisible.

-   *Floor allocation*: best if we need to compare across tools.

The final distribution for the double-coded sample is:

In [ ]:
set.seed(2025)

# Load the dataset
all_tools <- readRDS(here("05_cleaned_datasets", "all_tools.rds")) %>%
  janitor::clean_names() %>%
  mutate(
    year = as.integer(year),
    study_id = dplyr::row_number()
  )

# Parameters
TOTAL <- nrow(all_tools)               # should be 2212
SAMPLE_TOTAL <- ceiling(TOTAL * 0.10)  # 10% => 217
MIN_PER_TOOL <- 10                     # floor per tool
EXPAND_TOTAL <- TRUE                   # expand total if needed

# --------------------------------------------------------------------
# Allocation with per-tool minimums, then proportional distribution
# --------------------------------------------------------------------
tool_counts <- all_tools %>%
  count(tool_name, name = "n_tool") %>%
  arrange(desc(n_tool))

# Floors (min 10, capped at availability)
floor_alloc <- tool_counts %>%
  mutate(floor = pmin(n_tool, MIN_PER_TOOL))

reserved <- sum(floor_alloc$floor)

if (reserved > SAMPLE_TOTAL) {
  if (EXPAND_TOTAL) {
    SAMPLE_TOTAL <- reserved
  } else {
    stop("Minimum per-tool exceeds total sample. Lower MIN_PER_TOOL or allow expand.")
  }
}

remaining <- SAMPLE_TOTAL - reserved

# Proportional extras after floors
alloc <- floor_alloc %>%
  mutate(capacity = pmax(0, n_tool - floor))

if (remaining > 0 && sum(alloc$capacity) > 0) {
  alloc <- alloc %>%
    mutate(
      weight     = capacity / sum(capacity),
      extra_raw  = remaining * weight,
      extra_base = floor(extra_raw),
      remainder  = extra_raw - extra_base
    )
  leftover <- remaining - sum(alloc$extra_base)
  alloc <- alloc %>%
    arrange(desc(remainder)) %>%
    mutate(extra = extra_base + if_else(row_number() <= leftover, 1L, 0L)) %>%
    arrange(desc(n_tool))
} else {
  alloc <- alloc %>% mutate(extra = 0L)
}

# Final allocation
alloc <- alloc %>%
  mutate(alloc = pmin(n_tool, floor + extra))

tool_alloc <- alloc %>%
  select(tool_name, n_tool, alloc)

stopifnot(sum(tool_alloc$alloc) == SAMPLE_TOTAL)

# --------------------------------------------------------------------
# Stratified random sample
# --------------------------------------------------------------------
orig_cols <- names(all_tools)

sample_primary <- all_tools %>%
  inner_join(tool_alloc, by = "tool_name") %>%
  group_by(tool_name) %>%
  group_modify(~ dplyr::slice_sample(.x, n = unique(.x$alloc))) %>%
  ungroup() %>%
  mutate(
    double_code   = TRUE,
    topic_cat     = NA_character_,
    tool_use_type = NA_character_
  ) %>%
  select(all_of(orig_cols), double_code, topic_cat, tool_use_type)

stopifnot(nrow(sample_primary) == SAMPLE_TOTAL)

# --------------------------------------------------------------------
# Save 05_cleaned_datasetss (to 05_cleaned_datasets/ folder)
# --------------------------------------------------------------------
sample_primary %>% count(tool_name) %>% arrange(desc(n)) %>% print()

# A tibble: 5 × 2
  tool_name                                               n
  <chr>                                               <int>
1 EJScreen                                              111
2 Climate and Economic Justice Screening Tool (CEJST)    75
3 Environmental Justice Index (EJI)                      19
4 Climate Risk Viewer                                    11
5 Future Risk Index (FRI)                                 6

To support manual coding, I drafted an early codebook with candidate topic and tool use categories. Definitions, inclusion/exclusion criteria, and examples will be added as the coding scheme is refined.

In [ ]:
# Starter codebook for manual coding
topic_levels <- c(
  "Air quality / pollution",
  "Health impacts",
  "Extreme heat / climate risk",
  "Transportation",
  "Land use / housing",
  "Energy / utilities",
  "Water / flooding",
  "Equity / policy & governance"
)

tool_use_levels <- c(
  "Applied (used for analysis/targeting)",
  "Evaluated (validated/benchmarked the tool)",
  "Critiqued (limitations/methodological critique)",
  "Mentioned/Context (used for background)"
)

codebook <- tibble::tibble(
  variable   = c(rep("topic_cat", length(topic_levels)),
                 rep("tool_use_type", length(tool_use_levels))),
  code       = c(topic_levels, tool_use_levels),
  definition = NA_character_,   # fill in
  include_if = NA_character_,   # fill in
  exclude_if = NA_character_,   # fill in
  examples   = NA_character_    # fill in small examples/keywords
)

readr::write_csv(codebook, here("05_cleaned_datasets", "codebook_topic_tool_use.csv"))

### Inter-rater agreement

Before running the analysis, I’ll import two datasets that my colleagues and I collaborated on. These datasets include our manual categorizations of studies and their publishers.

**1. Study-level data**

-   `all_studies_ira.csv`: A collaborator and I manually categorized publishers for about **10% of the sample**. This dataset is used to calculate **inter-rater agreement (IRA)** and identify where our categorizations differed or aligned.
-   `all_studies_finalized.csv`: This is the **final dataset** after we reviewed disagreements and reached consensus. We updated the **codebook** accordingly, and applied the final categories to the full dataset. I will use this version for analyzing relevant research questions.
    -   The corresponding codebook, **`codebook_studies.csv`**, is located in the `codebooks/` folder.
    -   It defines **four categories** of study classification like applied, mentioned/context, etc. for a total of **10 subcategories.**

**2. Publisher-level data**

-   `all_publishers_ira.csv`: Similar to the studies dataset, a collaborator and I manually categorized about **11% of publishers**. This subset is used to assess IRA.
-   `all_publishers_finalized.csv`: The **finalized version** after resolving disagreements and updating the codebook. I will use this version for analyzing relevant research questions.
    -   The corresponding codebook, `codebook_publishers.csv`, is in the `codebooks/` folder.
    -   It defines **10 publisher types**, ranging from *commercial academic publishers* to *grey literature platforms*.

The first step in this analysis is to calculate **inter-rater agreement (IRA)** at both the study and publisher levels using **Cohen’s kappa**, a statistical measure of rater agreement.

I chose this approach because relying on a single person to categorize studies or publishers can introduce bias. With two collaborators independently code each dataset, I can identify where our classifications **converge** and **diverge**, and quantify that level of agreement.

Cohen’s kappa adjusts for agreement that occurs **by chance** and produces a value between **–1 and +1**, where –1 indicates no agreement and +1 indicates almost perfect agreement. In practice, it functions like a **correlation coefficient** based on a cross-tabulation of each rater’s classifications.

As McHugh (2012) explains, *“the squaring of kappa translates conceptually to the amount of accuracy (i.e., the reverse of error) in the data due to congruence among the data collectors.”*

**Interpretation of Cohen’s kappa** \[@mchugh2012\]

| Value of Kappa | Level of Agreement | % of Data that are Reliable |
|:--------------:|:------------------:|:---------------------------:|
|     0–.20      |        None        |            0–4%             |
|    .21–.39     |      Minimal       |            4–15%            |
|    .40–.59     |        Weak        |           15–35%            |
|    .60–.79     |      Moderate      |           35–63%            |
|    .80–.90     |       Strong       |           64–81%            |
|    Above.90    |   Almost Perfect   |           82–100%           |

A good resource that I’m following for IRA is *Cohen’s Kappa in R: For Two Categorical Variables* \[@datanovia2025\]. Another is the *agreement package:* <https://github.com/jmgirard/agreement>.

#### **1. Publisher-level data**

Cohen’s κ = 0.89 (95 % CI \[0.79, 0.99\], p \< 0.001) indicates **almost perfect agreement** between the two raters according to McHugh (2012). The raw percent agreement was 91 %, which higlights strong consistency between raters in categorizing publisher types.

Here’s how to read the results table:

| Metric | Value | Interpretation |
|---------------------|-------------------|--------------------------------|
| **Percent agreement** | `0.909` (≈ 91%) | Raw proportion of identical ratings — shows high agreement overall |
| **Unweighted Cohen’s κ** | `0.8935` | Agreement beyond chance |
| **Weighted Cohen’s κ** | `0.8602` | Takes into account partial agreement (useful for *ordinal* data — not critical here) |
| **ASE** (Asymptotic Std. Error) | `0.0505` | Measures uncertainty of κ estimate |
| **z** | `17.69` | Test statistic for H₀ : κ = 0 (no agreement beyond chance) |
| **p-value** | `4.78 × 10⁻⁷⁰` | Extremely small → reject H₀; the raters’ agreement is highly significant |
| **95% CI (Unweighted)** | 0.79 – 0.99 | Even the lower bound (0.79) indicates strong agreement |

**Note on sample size for inter-rater agreement** Although the full dataset (`all_publishers_ira.csv`) contains 387 records, only the first 44 rows were double-coded by both raters (11% of the sample).

The remaining rows include missing or unassigned categories for at least one rater. Because `table()` in R automatically excludes `NA` values when computing cross-tabulations, the Cohen’s Kappa and percent agreement calculations below are effectively based on those 44 complete cases.

So, this code can be reused with any dataset of similar structure without needing to hardcode a row range. The computation will always use only non-missing, valid pairs of rater classifications.

In [ ]:
# Import publisher file
all_publishers_ira <- readr::read_csv(
  here::here("02_processed_data", "all_publishers_ira.csv"),
  show_col_types = FALSE,
  na = c("", "NA", "N/A", "NULL")
)

# Enforce types
all_publishers_ira <- all_publishers_ira |>
  mutate(
    publisher        = as.character(publisher),
    n                = as.integer(n),
    pub_type_rater_1 = as.character(pub_type_rater_1),
    pub_type_rater_2 = as.character(pub_type_rater_2),
    description      = as.character(description)
  )

# Define category levels
kappa_levels <- c(
  "Commercial academic publishers",
  "University presses",
  "Open access publishers",
  "Preprint / working paper servers",
  "Government repositories",
  "Legal / policy databases",
  "University / institutional repositories",
  "Grey literature platforms",
  "Scholarly society / association publishers",
  "None/Unknown"
)

# Apply factor levels
all_publishers_ira <- all_publishers_ira |>
  mutate(
    pub_type_rater_1 = factor(pub_type_rater_1, levels = kappa_levels),
    pub_type_rater_2 = factor(pub_type_rater_2, levels = kappa_levels)
  )

# Use to check for the recoding results
# table(all_publishers_ira$pub_type_rater_1, useNA = "ifany")
# table(all_publishers_ira$pub_type_rater_2, useNA = "ifany")

# Build contingency table and compute Cohen's Kappa
# (Unweighted Kappa corresponds to Cohen’s kappa for two raters)
tab_10x10 <- table(
  Rater1 = all_publishers_ira$pub_type_rater_1,
  Rater2 = all_publishers_ira$pub_type_rater_2
)

kappa_res <- vcd::Kappa(tab_10x10)

kappa_res

            value     ASE     z  Pr(>|z|)
Unweighted 0.8935 0.05050 17.69 4.780e-70
Weighted   0.8602 0.07517 11.44 2.536e-30

            
Kappa              lwr      upr
  Unweighted 0.7944860 0.992439
  Weighted   0.7129097 1.000000

[1] 0.9090909

#### **2. Study-level data**

In [ ]:
# Import publisher file
sample_studies_ira <- readr::read_csv(
  here::here("02_processed_data", "double_code_sample_ira.csv"),
  show_col_types = FALSE,
  na = c("", "NA", "N/A", "NULL")
)

# Enforce types ONLY for IRA-relavent variables
sample_studies_ira <- sample_studies_ira %>%
  mutate(
    topic_cat_rater_1      = as.character(topic_cat_rater_1),
    topic_cat_rater_2      = as.character(topic_cat_rater_2),
    tool_use_type_rater_1  = as.character(tool_use_type_rater_1),
    tool_use_type_rater_2  = as.character(tool_use_type_rater_2)
  )

# Define factor levels
topic_levels <- c(
  "Air quality & pollution",
  "Health & well-being",
  "Climate hazards & risks",
  "Transit, mobility & transportation",
  "Land use, housing & built environment infrastructure",
  "Energy & decarbonization",
  "Water resources & flooding",
  "Natural environment, resources & ecosystems",
  "Governance, policy & institutional frameworks",
  "Equity and justice",
  "Economy, labor & development",
  "Community engagement & knowledge",
  "Other"
)

tool_use_levels <- c(
  "Applied",
  "Evaluated",
  "Critiqued",
  "Mentioned/Context",
  "None"
)

# Apply factor levels
sample_studies_ira <- sample_studies_ira %>%
  mutate(
    topic_cat_rater_1     = factor(topic_cat_rater_1,     levels = topic_levels),
    topic_cat_rater_2     = factor(topic_cat_rater_2,     levels = topic_levels),
    tool_use_type_rater_1 = factor(tool_use_type_rater_1, levels = tool_use_levels),
    tool_use_type_rater_2 = factor(tool_use_type_rater_2, levels = tool_use_levels)
  )

# Compute Cohen's kappa for TOPIC
tab_topic <- table(
  Rater1 = sample_studies_ira$topic_cat_rater_1,
  Rater2 = sample_studies_ira$topic_cat_rater_2
)
kappa_topic <- vcd::Kappa(tab_topic)
pa_topic <- mean(
  sample_studies_ira$topic_cat_rater_1 == sample_studies_ira$topic_cat_rater_2,
  na.rm = TRUE
)

# Compute Cohen's kappa for TOOL USE TYP
tab_tool <- table(
  Rater1 = sample_studies_ira$tool_use_type_rater_1,
  Rater2 = sample_studies_ira$tool_use_type_rater_2
)
kappa_tool <- vcd::Kappa(tab_tool)
pa_tool <- mean(
  sample_studies_ira$tool_use_type_rater_1 == sample_studies_ira$tool_use_type_rater_2,
  na.rm = TRUE
)

# Simple percent agreement for TOPIC
percent_agreement_topic <- mean(
  sample_studies_ira$topic_cat_rater_1 == sample_studies_ira$topic_cat_rater_2,
  na.rm = TRUE
)
percent_agreement_topic

[1] 0.3921569

[1] 0.3221154

            value     ASE     z  Pr(>|z|)
Unweighted 0.3179 0.03726 8.534 1.415e-17
Weighted   0.3338 0.05020 6.649 2.956e-11

            
Kappa              lwr       upr
  Unweighted 0.2449245 0.3909685
  Weighted   0.2353783 0.4321608

[1] 0.3921569

            value     ASE     z  Pr(>|z|)
Unweighted 0.1561 0.03340 4.673 2.966e-06
Weighted   0.2037 0.04214 4.834 1.336e-06

            
Kappa               lwr       upr
  Unweighted 0.09063148 0.2215718
  Weighted   0.12112305 0.2863082

[1] 0.3221154

# A tibble: 2 × 4
  variable      kappa_unweighted kappa_weighted percent_agreement
  <chr>                    <dbl>          <dbl>             <dbl>
1 topic_cat                0.318          0.334             0.392
2 tool_use_type            0.156          0.204             0.322

### Research questions

#### Research question 1

I will look at each question one by one and clean the data as I go. I will organize the data during the analysis before exploring the results. I’ll also export intermediate results into tidy CSV files so they are ready for further visualization and exploration.

How many research papers or studies have cited or relied on five U.S. federal climate and environmental tools that are threatened, not being maintained, or no longer available?

##### Total number of studies

In [ ]:
nrow (all_tools)

[1] 2212

##### Count of studies per tool

In [ ]:
df_tool <- all_tools %>%
  count(tool_name, name = "n_studies") %>%
  arrange(desc(n_studies))

hchart(df_tool, "column", hcaes(x = tool_name, y = n_studies)) %>%
  hc_title(text = "Number of studies by tool") %>%
  hc_subtitle(text = "Total count of research papers citing or relying on each tool") %>%
  hc_xAxis(title = list(text = "Tool")) %>%
  hc_yAxis(title = list(text = "Number of studies"), allowDecimals = FALSE) %>%
  hc_tooltip(pointFormat = "<b>{point.y}</b> studies") %>%
  hc_legend(enabled = FALSE) %>%
  hc_exporting(enabled = TRUE)

##### Count of studies per year per tool

In [ ]:
df_year_tool <- all_tools %>%
  filter(!is.na(year)) %>%
  count(tool_name, year, name = "n_studies") %>%
  arrange(tool_name, year)

hchart(
  df_year_tool,
  "line",
  hcaes(x = year, y = n_studies, group = tool_name)
) %>%
  hc_title(text = "Studies over time by tool") %>%
  hc_subtitle(text = "Annual counts of studies citing or relying on each tool") %>%
  hc_xAxis(title = list(text = "Year"), allowDecimals = FALSE) %>%
  hc_yAxis(title = list(text = "Number of studies"), allowDecimals = FALSE) %>%
  hc_tooltip(shared = TRUE, crosshairs = TRUE,
             headerFormat = "<b>Year: {point.key}</b><br/>",
             pointFormat = "{series.name}: <b>{point.y}</b><br/>") %>%
  hc_legend(title = list(text = "Tool")) %>%
  hc_exporting(enabled = TRUE)

##### Top 10 most cited studies overall per tool

In [ ]:
df_top_cites <- all_tools %>%
  filter(!is.na(cites)) %>%
  group_by(tool_name) %>%
  slice_max(order_by = cites, n = 10, with_ties = FALSE) %>%
  ungroup() %>%
  select(tool_name, year, title, cites, authors) %>%
  arrange(tool_name, desc(cites))

# Show table (optional)
df_top_cites

# A tibble: 46 × 5
   tool_name            year title                                 cites authors
   <chr>               <int> <chr>                                 <int> <chr>  
 1 Climate Risk Viewer  2024 Transition versus physical climate r…   234 G Bua,…
 2 Climate Risk Viewer    NA USDA Forest Service                      28 I Fore…
 3 Climate Risk Viewer  2024 Physical and transition risk premium…    15 JV Bat…
 4 Climate Risk Viewer  2024 Perceived climate risk and stock pri…    13 H Ben …
 5 Climate Risk Viewer  2023 Measuring the climate risk exposure …    13 H Jung…
 6 Climate Risk Viewer  2024 A practical framework for applied fo…     7 AD Bow…
 7 Climate Risk Viewer  2024 Physical climate risk factors and an…     6 H Jung…
 8 Climate Risk Viewer  2025 Land of opportunity: potential for r…     6 T Mai,…
 9 Climate Risk Viewer  2024 Climate risks, corporate bonds, and …     5 V Lalw…
10 Climate Risk Viewer  2023 Firm-specific climate risk estimated…     3 T Dang…
# ℹ 36 mo

##### Top 10 highest `cites_per_year` per tool

This shows which studies are “most cited relative to their age.”

In [ ]:
df_top_cpy <- all_tools %>%
  filter(!is.na(cites_per_year)) %>%
  group_by(tool_name) %>%
  slice_max(order_by = cites_per_year, n = 10, with_ties = FALSE) %>%
  ungroup()

df_top_cpy %>% 
  select(tool_name, year, title, cites_per_year, cites, authors) %>%
  arrange(tool_name, desc(cites_per_year))

# A tibble: 46 × 6
   tool_name            year title                  cites_per_year cites authors
   <chr>               <int> <chr>                           <dbl> <int> <chr>  
 1 Climate Risk Viewer  2024 Transition versus phy…          234     234 G Bua,…
 2 Climate Risk Viewer  2024 Physical and transiti…           15      15 JV Bat…
 3 Climate Risk Viewer  2024 Perceived climate ris…           13      13 H Ben …
 4 Climate Risk Viewer  2024 A practical framework…            7       7 AD Bow…
 5 Climate Risk Viewer  2023 Measuring the climate…            6.5    13 H Jung…
 6 Climate Risk Viewer  2024 Physical climate risk…            6       6 H Jung…
 7 Climate Risk Viewer  2025 Land of opportunity: …            6       6 T Mai,…
 8 Climate Risk Viewer  2024 Climate risks, corpor…            5       5 V Lalw…
 9 Climate Risk Viewer  2023 Firm-specific climate…            1.5     3 T Dang…
10 Climate Risk Viewer  2025 Climate Extrapolation…            1       1 G Verd…
# ℹ 36 mo

##### Average `cites_per_year` over time per tool

This shows whether more recent papers citing a tool are getting traction.

In [ ]:
df_avg_cpy <- all_tools %>%
  filter(!is.na(year), !is.na(cites_per_year)) %>%
  group_by(tool_name, year) %>%
  summarise(avg_cpy = mean(cites_per_year, na.rm = TRUE), .groups = "drop")

hchart(df_avg_cpy, "line", hcaes(x = year, y = avg_cpy, group = tool_name)) %>%
  hc_title(text = "Average cites per year over time, by tool") %>%
  hc_yAxis(title = list(text = "Average cites per year")) %>%
  hc_exporting(enabled = TRUE)

##### Publishers

Which top 10 publishers/platforms appear most frequently?

In [ ]:
df_10_publishers <- all_tools %>%
  filter(publisher != "") %>%
  count(publisher, sort = TRUE) %>%
  slice_head(n = 10)

df_10_publishers

              publisher   n
1   search.proquest.com 212
2              osti.gov 138
3            HeinOnline 127
4              Elsevier 118
5              Springer  88
6       papers.ssrn.com  61
7       Taylor &Francis  58
8        liebertpub.com  53
9  Wiley Online Library  46
10             mdpi.com  42

List all publishers from the sample.

In [ ]:
df_all_publishers <- all_tools %>%
  filter(publisher != "") %>%
  count(publisher, sort = TRUE)

write_csv(df_all_publishers, here("06_viz_ready", "all_publishers.csv"))

#### Research question 2

What research topics are associated with each tool, and how do these topics vary by citation frequency?

Coming soon!

#### Research question 3

In what ways are these tools applied in research?

Coming soon!